In [1]:
# imports
import pandas as pd
import numpy as np

In [ ]:
# Need to do some calculation for Y, S,T (use ratios for prime)
# purple group they in text --> not as useful --> not all of them are ID like features some of them might be useful 
    # 1 tend to be very useful 

In [ ]:
# CHECK HIGH AND LOW !!! --> INVALID VALUES
# TRULY MISSING VALUES??? CHECK
# are HIGH LOW CONNECTED TO NULL VALUES 
# WHAT DOES - mean --> when does it happen 

# high and low of the IPO
# Pmid fair estimation of P(H) or P(L) --> UNDERPRICING DETECTING (Y1, Y2)

In [2]:
data = pd.read_excel('Competition1_raw_data.xlsx')
                    #dtype={'P(IPO)': float, 'P(1Day)': float})
# define dashes as missing --> 0 or NaN
data.head(10)

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
0,AATI,ADVANCED ANALOGIC TECHNOLOGIES INC,3674,10,9.5,8.5,11.87,122,1,3.43,...,10600000,51.345,470,12719,11560,301,690,62,117,139
1,ABPI,ACCENTIA BIOPHARMACEUTICALS INC,2834,8,10,8,7.25,259,0,-1.62,...,2400000,25.936,791,21792,19585,510,1120,71,242,237
2,ACAD,ACADIA PHARMACEUTICALS INC,2834,7,14,12,6.7,90,1,-1.24,...,5000000,7.378,201,5262,4785,128,325,61,33,60
3,ACHN,ACHILLION PHARMACEUTICALS INC,2834,11.5,16,14,12.39,209,1,-0.91,...,4500000,8.526,328,8259,7574,177,509,80,59,110
4,ACLI,AMERICAN COMMERCIAL LINES INC.,4492,21,21,19,56.6,80,1,0.07,...,8250000,632.298,572,14830,13176,336,720,67,149,167
5,ACOM,Ancestry.com Inc.,7379,13.5,14.5,12.5,14.2,94,1,0.06,...,7407407,197.591,510,17999,13934,328,815,62,96,181
6,ACOR,ACORDA THERAPEUTICS INC,2836,6,13,11,6.72,128,1,-1.87,...,5500000,5.146,495,15758,13953,379,698,82,106,135
7,ACRX,ACELRX PHARMACEUTICALS INC,2834,5,14,12,4.55,91,0,-0.74,...,8000000,-,334,11226,10056,276,-1,55,120,122
8,ACTV,ACTIVE NETWORK INC,7372,15,18,16,15.9,100,1,-0.51,...,11000000,279.6,620,14813,13463,335,917,58,124,234
9,ACW,ACCURIDE CORP,3714,9,19,17,0,106,1,1.49,...,11000000,494.008,385,9134,8107,198,459,81,90,147


In [ ]:
data.info()

# PLAN
- In every stage make e.g. 5 different ways of doing that --> store that in a seperate notebook
- Connect pipelines in different ways (most probable)
- In the end have multiple notebooks/files to be tested 

# Data Cleaning
1. Finding Null/wrong values
    - '-' --> some 
    - dtype --> All 
    - -1 --> T5, S1
    - 0 (P(1Day)), T2, T3, T4
    - I3 industry type
    - P(H) < P(L)
    - C2 (how to fill that)
2. Make assumptions to why this happen and how to resolve/imputate it
    STEP1: Dashes (P(H), P(L)) --> those that have P(IPO) average distance 
    STEP2: find out what is the industry classifier (I3) --> that have dashes
    STEP3: infer average distance for P(1Day) --> cannot be more than 3x price --> mean distance 
    STEP4: C1 --> C7 --> those that have information give best possible guess to fill in the dash (e.g. average distance)
    STEP5: ASSUMTION -1 means none 
    STEP6: appropriate guess for 0 
    STEP6: data[data['P(H)'] < data['P(L)']] --> replace with best guess/switch  
    STEP7: Dashes --> replace all dashes with NaN
    
    STEP8: replace nan's with imputation method: Median, mode, mean

In [3]:
# Companies that have dashes in some places 
data[(data['P(1Day)']=='-') & (data['P(IPO)']=='-') & (data['P(H)']=='-')& (data['C1']=='-')& (data['C2']=='-')& 
     (data['C3']=='-')& (data['C4']=='-')& (data['C5']=='-')& (data['C6']=='-')& (data['C7']=='-')& (data['I3']=='-')]

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
10,ADKU,"ADCARE HEALTH SYSTEMS, INC",-,-,-,-,-,-,-,-,...,-,-,332,7534,6747,169,263,51,65,74
86,BLT,BLOUNT INTERNATIONAL INC,-,-,-,-,-,-,-,-,...,-,-,538,12732,11126,301,631,94,172,116
330,KFI,KKR Financial Holdings LLC,-,-,-,-,-,-,-,-,...,-,-,246,527,9046,355,10277,64,48,123
400,NAUH,"National American University Holdings, Inc.",-,-,-,-,-,-,-,-,...,-,-,290,8454,7137,192,384,45,68,139
597,TRMM,"Access to Money, Inc.",-,-,-,-,-,-,-,-,...,-,-,494,12004,10226,277,560,46,100,91


In [4]:
data['I3'].value_counts().head(16)

2834    76
7372    49
3674    36
7389    32
3841    30
1311    18
3845    14
7373    12
5812    11
7371    10
4412    10
7374     9
4899     9
2836     9
8731     9
-        8
Name: I3, dtype: int64

In [5]:
# I3 --> C7 
data.loc[data['P(IPO)'] == '-'] 

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
10,ADKU,"ADCARE HEALTH SYSTEMS, INC",-,-,-,-,-,-,-,-,...,-,-,332,7534,6747,169,263,51,65,74
86,BLT,BLOUNT INTERNATIONAL INC,-,-,-,-,-,-,-,-,...,-,-,538,12732,11126,301,631,94,172,116
330,KFI,KKR Financial Holdings LLC,-,-,-,-,-,-,-,-,...,-,-,246,527,9046,355,10277,64,48,123
400,NAUH,"National American University Holdings, Inc.",-,-,-,-,-,-,-,-,...,-,-,290,8454,7137,192,384,45,68,139
597,TRMM,"Access to Money, Inc.",-,-,-,-,-,-,-,-,...,-,-,494,12004,10226,277,560,46,100,91


In [6]:
# I3 --> C7 
data.loc[data['P(IPO)'] == 0] 

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3


In [7]:
data.loc[data['P(H)'] == '-'] 

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
10,ADKU,"ADCARE HEALTH SYSTEMS, INC",-,-,-,-,-,-,-,-,...,-,-,332,7534,6747,169,263,51,65,74
86,BLT,BLOUNT INTERNATIONAL INC,-,-,-,-,-,-,-,-,...,-,-,538,12732,11126,301,631,94,172,116
215,ESEA,EUROSEAS LTD.,4412,7,-,-,-,-,-,-,...,9602340,-,521,12812,11190,285,545,72,60,121
268,GSL,GLOBAL SIGNAL INC,-,18,-,-,-,-,-,-,...,7000000,-,631,19479,17409,451,976,71,252,155
330,KFI,KKR Financial Holdings LLC,-,-,-,-,-,-,-,-,...,-,-,246,527,9046,355,10277,64,48,123
400,NAUH,"National American University Holdings, Inc.",-,-,-,-,-,-,-,-,...,-,-,290,8454,7137,192,384,45,68,139
490,RCH,"CHINA CGAME, INC.",-,3.5,-,-,-,-,-,-,...,737000,-,322,9820,8893,230,410,61,92,91
560,STRL,STERLING CONSTRUCTION CO INC,1600,15,-,-,-,-,-,-,...,2021758,-,310,7195,6393,172,325,44,65,85
597,TRMM,"Access to Money, Inc.",-,-,-,-,-,-,-,-,...,-,-,494,12004,10226,277,560,46,100,91
614,TZOO,TRAVELZOO INC,-,3.5,-,-,-,-,-,-,...,-,-,146,3600,3189,88,181,5,26,38


In [8]:
data.loc[data['P(1Day)'] == '-'] 

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
10,ADKU,"ADCARE HEALTH SYSTEMS, INC",-,-,-,-,-,-,-,-,...,-,-,332,7534,6747,169,263,51,65,74
86,BLT,BLOUNT INTERNATIONAL INC,-,-,-,-,-,-,-,-,...,-,-,538,12732,11126,301,631,94,172,116
115,CELM,"CHINA ELECTRIC MOTOR, INC.",3621,4.5,6.5,5.5,-,-,-,-,...,5000000,-,351,10352,9296,235,482,52,124,120
123,CIL,"China Intelligent Lighting & Electronics, Inc.",3640,3,5,4,-,-,-,59.261,...,3350000,59.261,406,12065,11056,282,667,72,169,182
203,ELMD,"Electromed, Inc.",3845,4,6,4,-,-,-,12.999,...,1700000,12.999,224,6146,5551,154,386,29,52,140
215,ESEA,EUROSEAS LTD.,4412,7,-,-,-,-,-,-,...,9602340,-,521,12812,11190,285,545,72,60,121
217,EURX,EURAND N.V.,2834,16,19,17,-,-,-,109.336,...,7000000,109.336,-,-,-,-,-,-,-,-
268,GSL,GLOBAL SIGNAL INC,-,18,-,-,-,-,-,-,...,7000000,-,631,19479,17409,451,976,71,252,155
286,HPJ,"Highpower International, Inc.",3690,3.25,4,3,-,-,-,73.262,...,525000,73.262,377,10731,9703,240,486,45,104,96
330,KFI,KKR Financial Holdings LLC,-,-,-,-,-,-,-,-,...,-,-,246,527,9046,355,10277,64,48,123


## IMPUTING P(H)

In [9]:
data.loc[data['P(H)'] == '-']  #can impute the P(H) values that have an I(IPO) value---add 1

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
10,ADKU,"ADCARE HEALTH SYSTEMS, INC",-,-,-,-,-,-,-,-,...,-,-,332,7534,6747,169,263,51,65,74
86,BLT,BLOUNT INTERNATIONAL INC,-,-,-,-,-,-,-,-,...,-,-,538,12732,11126,301,631,94,172,116
215,ESEA,EUROSEAS LTD.,4412,7,-,-,-,-,-,-,...,9602340,-,521,12812,11190,285,545,72,60,121
268,GSL,GLOBAL SIGNAL INC,-,18,-,-,-,-,-,-,...,7000000,-,631,19479,17409,451,976,71,252,155
330,KFI,KKR Financial Holdings LLC,-,-,-,-,-,-,-,-,...,-,-,246,527,9046,355,10277,64,48,123
400,NAUH,"National American University Holdings, Inc.",-,-,-,-,-,-,-,-,...,-,-,290,8454,7137,192,384,45,68,139
490,RCH,"CHINA CGAME, INC.",-,3.5,-,-,-,-,-,-,...,737000,-,322,9820,8893,230,410,61,92,91
560,STRL,STERLING CONSTRUCTION CO INC,1600,15,-,-,-,-,-,-,...,2021758,-,310,7195,6393,172,325,44,65,85
597,TRMM,"Access to Money, Inc.",-,-,-,-,-,-,-,-,...,-,-,494,12004,10226,277,560,46,100,91
614,TZOO,TRAVELZOO INC,-,3.5,-,-,-,-,-,-,...,-,-,146,3600,3189,88,181,5,26,38


EUROSEAS LTD.
GLOBAL SIGNAL INC
CHINA CGAME, INC. 
STERLING CONSTRUCTION CO INC
TRAVELZOO INC

In [44]:
data = data.replace('-',np.nan)

In [13]:
data.loc[data['I2'] == 'EUROSEAS LTD.', 'P(H)'] = 8
data.loc[data['I2'] == 'EUROSEAS LTD.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3


In [14]:
data.loc[data['I2'] == 'GLOBAL SIGNAL INC', 'P(H)'] = 19
data.loc[data['I2'] == 'GLOBAL SIGNAL INC']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
268,GSL,GLOBAL SIGNAL INC,NaN,18.0,19.0,NaN,NaN,NaN,NaN,NaN,...,7000000.0,NaN,631.0,19479.0,17409.0,451.0,976.0,71.0,252.0,155.0


In [15]:
data.loc[data['I2'] == 'CHINA CGAME, INC.', 'P(H)'] = 4.5
data.loc[data['I2'] == 'CHINA CGAME, INC.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
490,RCH,"CHINA CGAME, INC.",NaN,3.5,4.5,NaN,NaN,NaN,NaN,NaN,...,737000.0,NaN,322.0,9820.0,8893.0,230.0,410.0,61.0,92.0,91.0


In [16]:
data.loc[data['I2'] == 'STERLING CONSTRUCTION CO INC', 'P(H)'] = 16
data.loc[data['I2'] == 'STERLING CONSTRUCTION CO INC']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
560,STRL,STERLING CONSTRUCTION CO INC,1600,15.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2021758.0,NaN,310.0,7195.0,6393.0,172.0,325.0,44.0,65.0,85.0


In [17]:
data.loc[data['I2'] == 'TRAVELZOO INC', 'P(H)'] = 4.5
data.loc[data['I2'] == 'TRAVELZOO INC']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
614,TZOO,TRAVELZOO INC,NaN,3.5,4.5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,146.0,3600.0,3189.0,88.0,181.0,5.0,26.0,38.0


## IMPUTING P(L)

In [18]:
data.loc[data['P(L)'] == np.nan] #distance between P(H)and P(L) is 2---P(L) must be lower

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3


In [19]:
data.loc[data['I2'] == 'EUROSEAS LTD.', 'P(L)'] = 6
data.loc[data['I2'] == 'EUROSEAS LTD.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3


In [20]:
data.loc[data['I2'] == 'GLOBAL SIGNAL INC', 'P(L)'] = 17
data.loc[data['I2'] == 'GLOBAL SIGNAL INC']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
268,GSL,GLOBAL SIGNAL INC,NaN,18.0,19.0,17.0,NaN,NaN,NaN,NaN,...,7000000.0,NaN,631.0,19479.0,17409.0,451.0,976.0,71.0,252.0,155.0


In [21]:
data.loc[data['I2'] == 'CHINA CGAME, INC.', 'P(L)'] = 2.5
data.loc[data['I2'] == 'CHINA CGAME, INC.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
490,RCH,"CHINA CGAME, INC.",NaN,3.5,4.5,2.5,NaN,NaN,NaN,NaN,...,737000.0,NaN,322.0,9820.0,8893.0,230.0,410.0,61.0,92.0,91.0


In [22]:
data.loc[data['I2'] == 'STERLING CONSTRUCTION CO INC', 'P(L)'] = 14
data.loc[data['I2'] == 'STERLING CONSTRUCTION CO INC']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
560,STRL,STERLING CONSTRUCTION CO INC,1600,15.0,16.0,14.0,NaN,NaN,NaN,NaN,...,2021758.0,NaN,310.0,7195.0,6393.0,172.0,325.0,44.0,65.0,85.0


In [23]:
data.loc[data['I2'] == 'TRAVELZOO INC', 'P(L)'] = 2.5
data.loc[data['I2'] == 'TRAVELZOO INC']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
614,TZOO,TRAVELZOO INC,NaN,3.5,4.5,2.5,NaN,NaN,NaN,NaN,...,NaN,NaN,146.0,3600.0,3189.0,88.0,181.0,5.0,26.0,38.0


In [24]:
pd.set_option('display.max_rows', None)

## IMPUTING P(1Day)

In [ ]:
#distance between P(IPO) and P(1Day)---0.89--- add 0.89 to P(IPO) to get value

In [25]:
data.loc[data['I2'] == 'CHINA ELECTRIC MOTOR, INC.', 'P(1Day)'] = 5.39
data.loc[data['I2'] == 'CHINA ELECTRIC MOTOR, INC.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
115,CELM,"CHINA ELECTRIC MOTOR, INC.",3621,4.5,6.5,5.5,5.39,NaN,NaN,NaN,...,5000000.0,NaN,351.0,10352.0,9296.0,235.0,482.0,52.0,124.0,120.0


In [26]:
data.loc[data['I2'] == 'China Intelligent Lighting & Electronics, Inc.', 'P(1Day)'] = 3.89
data.loc[data['I2'] == 'China Intelligent Lighting & Electronics, Inc.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
123,CIL,"China Intelligent Lighting & Electronics, Inc.",3640,3.0,5.0,4.0,3.89,NaN,NaN,59.261,...,3350000.0,59.261,406.0,12065.0,11056.0,282.0,667.0,72.0,169.0,182.0


In [27]:
data.loc[data['I2'] == 'Electromed, Inc.', 'P(1Day)'] = 4.89
data.loc[data['I2'] == 'Electromed, Inc.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3


In [28]:
data.loc[data['I2'] == 'EUROSEAS LTD.', 'P(1Day)'] = 7.89
data.loc[data['I2'] == 'EUROSEAS LTD.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3


In [29]:
data.loc[data['I2'] == 'EURAND N.V.', 'P(1Day)'] = 16.89
data.loc[data['I2'] == 'EURAND N.V.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3


In [30]:
data.loc[data['I2'] == 'GLOBAL SIGNAL INC', 'P(1Day)'] = 18.89
data.loc[data['I2'] == 'GLOBAL SIGNAL INC']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
268,GSL,GLOBAL SIGNAL INC,NaN,18.0,19.0,17.0,18.89,NaN,NaN,NaN,...,7000000.0,NaN,631.0,19479.0,17409.0,451.0,976.0,71.0,252.0,155.0


In [31]:
data.loc[data['I2'] == 'Highpower International, Inc.', 'P(1Day)'] = 4.14
data.loc[data['I2'] == 'Highpower International, Inc.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
286,HPJ,"Highpower International, Inc.",3690,3.25,4.0,3.0,4.14,NaN,NaN,73.262,...,525000.0,73.262,377.0,10731.0,9703.0,240.0,486.0,45.0,104.0,96.0


In [32]:
data.loc[data['I2'] == 'Lizhan Environmental Corp', 'P(1Day)'] = 4.89
data.loc[data['I2'] == 'Lizhan Environmental Corp']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3


In [33]:
data.loc[data['I2'] == 'NovaBay Pharmaceuticals, Inc.', 'P(1Day)'] = 4.89
data.loc[data['I2'] == 'NovaBay Pharmaceuticals, Inc.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
401,NBY,"NovaBay Pharmaceuticals, Inc.",2834,4.0,6.0,4.0,4.89,NaN,NaN,1.533,...,5000000.0,1.533,382.0,9959.0,9215.0,237.0,534.0,73.0,88.0,132.0


In [34]:
data.loc[data['I2'] == 'NIVS IntelliMedia Technology Group, Inc.', 'P(1Day)'] = 4.39
data.loc[data['I2'] == 'NIVS IntelliMedia Technology Group, Inc.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
410,NIV,"NIVS IntelliMedia Technology Group, Inc.",3651,3.5,4.0,3.25,4.39,NaN,NaN,101.048,...,550000.0,101.048,451.0,12594.0,11210.0,298.0,565.0,63.0,115.0,170.0


In [35]:
data.loc[data['I2'] == 'CHINA CGAME, INC.', 'P(1Day)'] = 4.39
data.loc[data['I2'] == 'CHINA CGAME, INC.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
490,RCH,"CHINA CGAME, INC.",NaN,3.5,4.5,2.5,4.39,NaN,NaN,NaN,...,737000.0,NaN,322.0,9820.0,8893.0,230.0,410.0,61.0,92.0,91.0


In [36]:
data.loc[data['I2'] == 'Regulus Therapeutics Inc.', 'P(1Day)'] = 4.89
data.loc[data['I2'] == 'Regulus Therapeutics Inc.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
496,RGLS,Regulus Therapeutics Inc.,2834,4.0,12.0,10.0,4.89,NaN,NaN,13.789,...,11250000.0,13.789,344.0,11091.0,10333.0,258.0,677.0,129.0,87.0,154.0


In [37]:
data.loc[data['I2'] == 'CREATIVE REALITIES,', 'P(1Day)'] = 4.89
data.loc[data['I2'] == 'CREATIVE REALITIES,']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
503,RNIN,"CREATIVE REALITIES,",7373,4.0,5.0,4.0,4.89,NaN,NaN,710.216,...,4500000.0,710.216,199.0,5073.0,4395.0,95.0,249.0,27.0,56.0,26.0


In [38]:
data.loc[data['I2'] == 'STERLING CONSTRUCTION CO INC,', 'P(1Day)'] = 15.89
data.loc[data['I2'] == 'STERLING CONSTRUCTION CO INC,']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3


In [39]:
data.loc[data['I2'] == 'TRAVELZOO INC', 'P(1Day)'] = 4.39
data.loc[data['I2'] == 'TRAVELZOO INC']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
614,TZOO,TRAVELZOO INC,NaN,3.5,4.5,2.5,4.39,NaN,NaN,NaN,...,NaN,NaN,146.0,3600.0,3189.0,88.0,181.0,5.0,26.0,38.0


In [40]:
data.loc[data['I2'] == 'Ocera Therapeutics, Inc.', 'P(1Day)'] = 4.89
data.loc[data['I2'] == 'Ocera Therapeutics, Inc.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3


In [41]:
data.loc[data['I2'] == 'ZOGENIX, INC.', 'P(1Day)'] = 4.89
data.loc[data['I2'] == 'ZOGENIX, INC.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3
675,ZGNX,"ZOGENIX, INC.",2834,4.0,14.0,12.0,4.89,NaN,NaN,NaN,...,14000000.0,NaN,518.0,16898.0,15522.0,370.0,790.0,71.0,146.0,190.0


In [43]:
data.loc[data['P(L)'] == '-'] 

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,C6,C7,T1,T2,T3,T4,T5,S1,S2,S3


### ASSUMPTION P(H) is generally higher than P(IPO)

In [45]:
data = data.replace('-',np.nan)
data['IPO/H_dist'] = data['P(H)'] - data['P(IPO)'] 
data['IPO/H_dist'].median()

1.0

In [46]:
data['IPO/L_dist'] = data['P(IPO)'] - data['P(L)']
data['IPO/L_dist'].median()

0.5

In [47]:
data['H/L_dist'] = data['P(H)'] - data['P(L)'] 
data['H/L_dist'].median()

2.0

## ASSUMPTION P(1Day) usually larger 

In [48]:
data['P(IPO)/P(1Day)_dist'] = data['P(1Day)'] - data['P(IPO)'] 
data['P(IPO)/P(1Day)_dist'].median()

0.8900000000000001

# I3 dashes and zereos

In [49]:
data['I2'] = data['I2'].str.strip()

In [50]:
data.loc[data['I2'] == 'CHINA CGAME, INC.', 'I3'] = 1600
data.loc[data['I2'] == 'CHINA CGAME, INC.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,T3,T4,T5,S1,S2,S3,IPO/H_dist,IPO/L_dist,H/L_dist,P(IPO)/P(1Day)_dist
490,RCH,"CHINA CGAME, INC.",1600,3.5,4.5,2.5,4.39,NaN,NaN,NaN,...,8893.0,230.0,410.0,61.0,92.0,91.0,1.0,1.0,2.0,0.89


In [51]:
data.loc[data['I2'] == 'Access to Money, Inc.', 'I3'] = 7011
data.loc[data['I2'] == 'Access to Money, Inc.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,T3,T4,T5,S1,S2,S3,IPO/H_dist,IPO/L_dist,H/L_dist,P(IPO)/P(1Day)_dist
597,TRMM,"Access to Money, Inc.",7011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10226.0,277.0,560.0,46.0,100.0,91.0,NaN,NaN,NaN,NaN


In [52]:
data.loc[data['I2'] == 'BLOUNT INTERNATIONAL INC', 'I3'] = 1600
data.loc[data['I2'] == 'BLOUNT INTERNATIONAL INC']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,T3,T4,T5,S1,S2,S3,IPO/H_dist,IPO/L_dist,H/L_dist,P(IPO)/P(1Day)_dist
86,BLT,BLOUNT INTERNATIONAL INC,1600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11126.0,301.0,631.0,94.0,172.0,116.0,NaN,NaN,NaN,NaN


In [53]:
data.loc[data['I2'] == 'GLOBAL SIGNAL INC', 'I3'] = 3577
data.loc[data['I2'] == 'GLOBAL SIGNAL INC']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,T3,T4,T5,S1,S2,S3,IPO/H_dist,IPO/L_dist,H/L_dist,P(IPO)/P(1Day)_dist
268,GSL,GLOBAL SIGNAL INC,3577,18.0,19.0,17.0,18.89,NaN,NaN,NaN,...,17409.0,451.0,976.0,71.0,252.0,155.0,1.0,1.0,2.0,0.89


In [54]:
data.loc[data['I2'] == 'National American University Holdings, Inc.', 'I3'] = 8221
data.loc[data['I2'] == 'National American University Holdings, Inc.']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,T3,T4,T5,S1,S2,S3,IPO/H_dist,IPO/L_dist,H/L_dist,P(IPO)/P(1Day)_dist
400,NAUH,"National American University Holdings, Inc.",8221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7137.0,192.0,384.0,45.0,68.0,139.0,NaN,NaN,NaN,NaN


In [55]:
data.loc[data['I2'] == 'ADCARE HEALTH SYSTEMS, INC', 'I3'] = 8099
data.loc[data['I2'] == 'ADCARE HEALTH SYSTEMS, INC']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,T3,T4,T5,S1,S2,S3,IPO/H_dist,IPO/L_dist,H/L_dist,P(IPO)/P(1Day)_dist
10,ADKU,"ADCARE HEALTH SYSTEMS, INC",8099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6747.0,169.0,263.0,51.0,65.0,74.0,NaN,NaN,NaN,NaN


In [56]:
data.loc[data['I2'] == 'TRAVELZOO INC', 'I3'] = 7374
data.loc[data['I2'] == 'TRAVELZOO INC']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,T3,T4,T5,S1,S2,S3,IPO/H_dist,IPO/L_dist,H/L_dist,P(IPO)/P(1Day)_dist
614,TZOO,TRAVELZOO INC,7374,3.5,4.5,2.5,4.39,NaN,NaN,NaN,...,3189.0,88.0,181.0,5.0,26.0,38.0,1.0,1.0,2.0,0.89


In [57]:
data.loc[data['I2'] == 'KKR Financial Holdings LLC', 'I3'] = 7011
data.loc[data['I2'] == 'KKR Financial Holdings LLC']

,I1,I2,I3,P(IPO),P(H),P(L),P(1Day),C1,C2,C3,...,T3,T4,T5,S1,S2,S3,IPO/H_dist,IPO/L_dist,H/L_dist,P(IPO)/P(1Day)_dist
330,KFI,KKR Financial Holdings LLC,7011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9046.0,355.0,10277.0,64.0,48.0,123.0,NaN,NaN,NaN,NaN


CHINA CGAME, INC.	0.0 --> Arhitecture --> 1600
Access to Money, Inc.	0.0 --> 7011.0
BLOUNT INTERNATIONAL INC	 1600
GLOBAL SIGNAL INC	0.0 --> 3577.0
National American University Holdings, Inc.	0.0 --> 8221.0
ADCARE HEALTH SYSTEMS, INC	 8099
TRAVELZOO INC	0.0 --> 7374.0
KKR Financial Holdings LLC	 7011.0

174	Dolby Laboratories, Inc.	 3672.0
187	DTS, INC.	3672.0
499	RealD Inc.	 3672.0
620	UNIVERSAL POWER GROUP	3674.0

In [ ]:
data.head(12) # see changes to I3 in only selected rows

In [ ]:
i2_i3 = data[['I2','I3']]
i2_i3 = i2_i3.sort_values('I3')
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', None)

In [ ]:
data['I3'] = data['I3'].replace('-',np.nan)
data['I3'] = data['I3'].str.strip()
data['I3'] = pd.to_numeric(data['I3'], errors='coerce')

In [ ]:
data.loc[data['I3'] == '-'] 

In [ ]:
data.head(10)

In [ ]:
data.loc[data['C1'] == '-'] 

In [ ]:
# -1 values in data frame
data[data['T5']==-1]

In [ ]:
# -1 values in data frame
data[data['S1']==-1]

In [ ]:
# 0 values 
data[data['P(H)']==0]

In [ ]:
# 0 values 
data[data['T2']==0]

In [ ]:
# 0 values 
data[data['T3']==0]

In [ ]:
# 0 values 
data[data['T4']==0]

In [ ]:
# 0 values 
data[data['C2']==0]

In [ ]:
# 0 values 
data[data['C3']==0]

In [ ]:
# 0 values 
data[data['P(1Day)']==0]

In [ ]:
# P(H) < P(L)
data[data['P(H)'] < data['P(L)']]

# Imputation
### Decide on the order + different pipelines 
- replace dashes (-), 0, -1, P(H) < P(L) with NaN
- number and % of missing data<br>
    - percent_missing = df.isnull().sum() * 100 / len(df)<br>
    missing_value_df = pd.DataFrame({'column_name': df.columns,'percent_missing': percent_missing})
- make them a seperate columns 

- correct dtype<br>
- column calculation 
    - the problem probably was because we made calculations for new columns before imputing (so first impute then calculate new columns)

Methods:
1. bbfill 
2. ffill
3. median
4. mean
5. mode
6. Treating missing as seperate value (new column/binary value)
7. Infer Baysen/classification tree's
    - http://nadbordrozd.github.io/blog/2017/03/05/missing-data-imputation-with-bayesian-networks/
    - https://pypi.org/project/predictive_imputer/
    - https://www.analyticsvidhya.com/blog/2016/03/tutorial-powerful-packages-imputing-missing-values/
    - https://xgboost.readthedocs.io/en/latest/
    - 
8. Binning??

# Descriptive 
1. https://www.dataquest.io/blog/basic-statistics-with-python-descriptive-statistics/
- .describe()
- scipy.stats.describe(y, ddof=1, bias=False)
- https://realpython.com/python-statistics/


Some questions we could also answer: (NOT REQUIRED)
- how did companies do/were rated based on their EPS?
- does it have any effect on a company depending on the number of days its in the IPO process 
- does higher IPO price offer better performance/outlook
- in what condition the company is (EPS, earnings...) based on how many shares they sell
- does underwriter rating have any effect and is it correlated with positive/neg words, word lenght etc.
- Lower price more shares? higher earnings less shares?

# CORRELATION and simple plots 
- https://realpython.com/python-statistics/ (Towards the bottom)
- also in lectures that professors posted

# Normality Test 
- https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/

- Histogram
- Quantile-Quantile Plot
- Statistical Normality Tests
- Shapiro-Wilk Test

# Normalization

# Standardization

# Discretization 

# Conversion

# Outliers 

# Data Reduction

# Extra

In [ ]:
# first count how many 0's
data = data.astype(str)
data['P(1Day)'] = data['P(1Day)'].str.strip()
data['P(1Day)'] = data['P(1Day)'].replace('-', 0)
data['P(1Day)'] = data['P(1Day)'].astype(float)

In [ ]:
data = data.astype(str)
data = data.replace('-',0)
data[['P(IPO)','P(H)','P(L)','P(1Day)','C1','C3','C4','C5','C6','C7','T1','T2','T3','T4','T5','S1','S2','S3']] = data[['P(IPO)','P(H)','P(L)','P(1Day)','C1','C3','C4','C5','C6','C7','T1','T2','T3','T4','T5','S1','S2','S3']].astype(float)

In [ ]:
data['C2'] = data['C2'].astype(int)

In [ ]:
data.info()

In [ ]:
# 
data['T2'].value_counts()

In [ ]:
# similarities between different null values 
sim = data[data['P(1Day)']==0]

P(mid) = (P(H)+P(L))/2

In [ ]:
# P(mid) calculation and add it to the column 
data['P(mid)'] = (data['P(H)'] - data['P(L)'])/2

In [ ]:
data.info()

# this will be different after imputation

In [ ]:
# targets to be calculated
# make it equal to P(IPO) then we will just replace
data['Y1'] = data['P(IPO)']
data['Y2'] = data['P(IPO)']

# replace Y1 and change dtype to binary
data.loc[(data['Y1'] < data['P(mid)']),'Y1'] = 1
data.loc[(data['Y1'] > data['P(mid)']),'Y1'] = 0
data['Y1'] = data['Y1'].astype(int)

# replace Y2
data.loc[(data['Y2'] < data['P(1Day)']),'Y2'] = 1
data.loc[(data['Y2'] > data['P(1Day)']),'Y2'] = 0
data['Y2'] = data['Y2'].astype(int)

# Control values to be calculated
data['C3`'] = data['C3']
data['C3`'] = np.where((data['C3'] > 0),1,data['C3`'])
data['C3`'] = np.where((data['C3'] < 0),0,data['C3`'])
data['C3`'] = data['C3`'].astype(int)

data['C5`'] = data['C5']/data['C6']


# % SIGN??
data['C6calc'] = ((data['P(IPO)'] - data['P(mid)']) / data['P(mid)']) * 1
data['C6`'] = 0
data['C6`'] = np.where((data['P(IPO)'] > data['P(mid)']),data['C6calc'],data['C6`'])
data['C6`'] = np.where((data['P(IPO)'] < data['P(mid)']),0,data['C6`'])

data.head(20)

In [ ]:
sim_missing_val = data[(data['P(1Day)']==0) & (data['P(IPO)']==0) & (data['P(H)']==0)].count()

In [ ]:
# number of zeros in each column 
(data == 0).astype(int).sum(axis=0)

In [ ]:
# Number of non-zero values 
data.astype(bool).sum(axis=0)

In [ ]:
data.head()

In [ ]:
# replace with nulls
# maybe we can run it with zeros/np.nan
cols = ["P(IPO)","P(H)","P(L)","P(1Day)","C1",'C2','C3','C4','C5','C6','C7','T1','T2','T3','T4','T5','S1','S2','S3']
data[cols] = data[cols].replace({0:np.nan})

# Imputate with median
# check if it works right 
data[cols]=data[cols].fillna(data[cols].median())
#.iloc[0]
data['C5'] = data['C5'].fillna(data['C5'].median())

# display
data.head(12)

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
import matplotlib.pyplot as plt
plt.hist(data['P(IPO)'])
plt.show()

In [ ]:
pd.plotting.scatter_matrix(data[["P(IPO)","P(H)","P(L)"]], figsize=(14,14), hist_kwds={'bins':8}, alpha=.5, marker='o', s=50)

In [ ]:
no_object = data.select_dtypes(exclude=['object'])

In [ ]:
no_object.values

In [ ]:
# normalize with min/max
x = no_object.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

In [ ]:
# normalization with sklearn 
from sklearn import preprocessing
# Normalize P(IPO) column (1 column)
x_array = np.array(data['P(IPO)'])
normalized_X = preprocessing.normalize([x_array])

In [ ]:
normalized_X

In [ ]:
# mean normalization
normalized_df=(df-df.mean())/df.std()

# min/max normalization 
normalized_df=(df-df.min())/(df.max()-df.min())

# Note that `select_dtypes` returns a data frame. We are selecting only the columns
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols].apply(zscore)

# Standardization/normalization 

In [ ]:
# STANDARDIZATION
# Get column names first
names = df.columns
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_df, columns=names)

In [ ]:
# Correlation Analysis 

In [ ]:
# WHEN YOU HAVE ALL THE CALCULATIONS AND VARIATIONS DOWN DECIDE ON THE ORDER OF THE PROCESSING AND WHAT TO TRY AFTER
# THEN RUN THE MODEL